In [48]:
#import statements
import pandas as pd 
import numpy as np 
import pickle
pd.set_option('display.max_columns', None)

**Find the data codebook here**:   https://metadata.phila.gov/#home/datasetdetails/5543865f20583086178c4ee5/representationdetails/55d624fdad35c7e854cb21a4/?view_287_page=1

**Columns to keep**:   'assessment_date', 'basements', 
                 'category_code_description','census_tract', 'central_air','depth',
                 'exterior_condition', 'fireplaces', 'frontage', 'fuel', 'garage_spaces',
                 'geographic_ward', 
                 'interior_condition',
                 'market_value', 'market_value_date', 'number_of_bathrooms', 
                 'number_of_bedrooms', 'number_of_rooms','number_stories', 'quality_grade', 
                 'sale_date', 'sale_price', 
                 'topography', 'total_area','total_livable_area', 'type_heater', 
                 'unfinished',  'view_type', 'year_built',
                 'zip_code',  'lat', 'lng'

In [49]:
#place columns of interest into a list
cols_interest = ['assessment_date', 'basements', 
                 'category_code_description','census_tract', 'central_air','depth',
                 'exterior_condition', 'fireplaces', 'frontage', 'fuel', 'garage_spaces',
                 'geographic_ward', 
                 'interior_condition',
                 'market_value', 'market_value_date', 'number_of_bathrooms', 
                 'number_of_bedrooms', 'number_of_rooms','number_stories', 'quality_grade', 
                 'sale_date', 'sale_price', 
                 'topography', 'total_area','total_livable_area', 'type_heater', 
                 'unfinished',  'view_type', 'year_built',
                 'zip_code',  'lat', 'lng']

In [50]:
#read in data using columns of interest 
#query only those multifamily and single family homes
dat = (pd.read_csv("/home/jovyan/work/Philadelphia-Housing/processing/opa_properties_public.csv", 
                 usecols = cols_interest)
       .query('category_code_description == "Multi Family" | category_code_description == "Single Family"')
)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (67,71) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
dat['basements'] = dat['basements'].map({'A': 'full', 'B': 'full','C': 'full','D': 'full','I': 'full',
                                        'E': 'partial','F': 'partial','G': 'partial','H': 'partial','J': 'partial',
                                        '0': 'None'})
dat['central_air'] = dat['central_air'].map({'0': 'N', '1': 'Y','Y': 'Y', 'N': 'N'})

dat['fuel'] = dat['fuel'].map({'A': 'NG', 'B': 'Oil','C': 'Electric',
                              'E': 'other', 'G': 'other','H': 'other','I':'other'})
dat['topography'] = dat['topography'].map({'A': 'A', 'B': 'B','C': 'C',
                              'D': 'D', 'E': 'E','F': 'F'})

dat['type_heater'] = dat['type_heater'].map({'A': 'A', 'B': 'B','C': 'C',
                              'D': 'D', 'E': 'E','G': 'G','H': 'H'})

Below, we output the percentage missing by column:  
Note that assessment_date, fuel, market_value_date, quality_grade, and unfinished are all over ~90% missing.   
For these reasons, we will drop these columns from the analysis, as they will not provide robust and useful information for the analysis and prediction process.  

In [52]:
dat.isnull().sum() * 100 / len(dat)

assessment_date               93.949851
basements                     36.815857
category_code_description      0.000000
census_tract                   0.006938
central_air                   44.558375
depth                          0.077899
exterior_condition             0.088206
fireplaces                     0.164718
frontage                       0.079088
fuel                          97.111992
garage_spaces                  0.199405
geographic_ward                0.006938
interior_condition             0.109812
market_value                   0.018236
market_value_date            100.000000
number_of_bathrooms            0.143707
number_of_bedrooms             0.083845
number_of_rooms                5.908622
number_stories                 0.083251
quality_grade                 89.448167
sale_date                      0.001586
sale_price                     0.002180
topography                     6.621011
total_area                     0.047572
total_livable_area             0.038454


Next, we will remove those datapoints/rows that do not have values for **sale price** or **sale date**.   
These are removed since these are our outcome variables.  
Both the sales prices and the date which the house was sold is necessary to perform our experiment.  

In [53]:
#drop the rows which 'sale_date' is NAN
dat = dat[dat['sale_date'].notna()]
#drop the rows which 'sale_price' is NAN
dat = dat[dat['sale_price'].notna()]

In [54]:
dat = dat.drop(['assessment_date', 'fuel', 
         'market_value_date', 'quality_grade', 
         'unfinished'], axis = 1)

In [55]:
(dat['sale_price'] > 5_000_000).mean() *100

0.2820675971131184

*Note*: 0.28% of the data has a sale_price of greater than $5 million.   
We remove these from the dataset since we are not interested in these homes.  

In [56]:
dat = dat[dat['sale_price'] < 5_000_000]

In [57]:
dat.head()

,basements,category_code_description,census_tract,central_air,depth,exterior_condition,fireplaces,frontage,garage_spaces,geographic_ward,interior_condition,market_value,number_of_bathrooms,number_of_bedrooms,number_of_rooms,number_stories,sale_date,sale_price,topography,total_area,total_livable_area,type_heater,view_type,year_built,zip_code,lat,lng
0,full,Single Family,139.0,Y,94.0,1.0,NaN,16.0,NaN,29.0,1.0,263000.0,NaN,3.0,10.0,1.0,2021-08-06 00:00:00,263000.0,F,1542.0,1242.0,A,NaN,2019,19121.0,-75.174210,39.979126
1,None,Single Family,8.0,Y,0.0,3.0,0.0,0.0,0.0,8.0,3.0,309800.0,0.0,0.0,NaN,1.0,2021-07-16 00:00:00,272500.0,F,0.0,524.0,NaN,C,1900,19103.0,-75.170423,39.949254
2,NaN,Single Family,160.0,Y,50.0,1.0,NaN,NaN,NaN,31.0,1.0,660000.0,2.0,3.0,NaN,3.0,2021-07-06 00:00:00,655000.0,F,800.0,2499.0,A,I,2020,19125.0,-75.128893,39.978672
3,full,Single Family,216.0,Y,109.0,1.0,0.0,80.0,NaN,21.0,1.0,570400.0,2.0,3.0,6.0,2.0,2021-08-10 00:00:00,605950.0,F,8219.0,2716.0,A,I,2021,19128.0,-75.235043,40.045622
9,NaN,Multi Family,215.0,NaN,38.0,3.0,0.0,68.0,0.0,21.0,3.0,202600.0,0.0,0.0,NaN,2.0,2021-11-09 00:00:00,1.0,F,4712.0,2356.0,NaN,I,1940,19127.0,-75.229891,40.029924


We then pickle the dataset for later use and easy loading:    

In [58]:
pickle_out = open("housedat.pickle","wb")
pickle.dump(dat, pickle_out)
pickle_out.close()

possible predictions datasets:   
https://www.kaggle.com/datasets/harlfoxem/housesalesprediction     
https://github.com/michellesklee/predicting_home_values    